In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv("Amazon.csv")

# Show first few rows
print(df.head())

# Show summary info
print(df.info())

# Check basic statistics
print(df.describe())


      OrderID   OrderDate  CustomerID   CustomerName ProductID  \
0  ORD0000001  2023-01-31  CUST001504  Vihaan Sharma    P00014   
1  ORD0000002  2023-12-30  CUST000178    Pooja Kumar    P00040   
2  ORD0000003  2022-05-10  CUST047516    Sneha Singh    P00044   
3  ORD0000004  2023-07-18  CUST030059   Vihaan Reddy    P00041   
4  ORD0000005  2023-02-04  CUST048677  Aditya Kapoor    P00029   

           ProductName        Category       Brand  Quantity  UnitPrice  \
0           Drone Mini           Books   BrightLux         3     106.59   
1           Microphone  Home & Kitchen  UrbanStyle         1     251.37   
2  Power Bank 20000mAh        Clothing  UrbanStyle         3      35.03   
3       Webcam Full HD  Home & Kitchen      Zenith         5      33.58   
4              T-Shirt        Clothing    KiddoFun         2     515.64   

   Discount    Tax  ShippingCost  TotalAmount     PaymentMethod OrderStatus  \
0      0.00   0.00          0.09       319.86        Debit Card   Deliver

In [3]:
# Count missing values per column
print(df.isnull().sum())

# Fill or drop missing values as needed
df = df.dropna(subset=['OrderID','CustomerID','ProductID'])


OrderID          0
OrderDate        0
CustomerID       0
CustomerName     0
ProductID        0
ProductName      0
Category         0
Brand            0
Quantity         0
UnitPrice        0
Discount         0
Tax              0
ShippingCost     0
TotalAmount      0
PaymentMethod    0
OrderStatus      0
City             0
State            0
Country          0
SellerID         0
dtype: int64


In [8]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce')
print(df['OrderDate'])

0       2023-01-31
1       2023-12-30
2       2022-05-10
3       2023-07-18
4       2023-02-04
           ...    
99995   2023-03-07
99996   2021-11-24
99997   2023-04-29
99998   2021-11-01
99999   2021-12-04
Name: OrderDate, Length: 100000, dtype: datetime64[ns]


In [11]:
df['Year'] = df['OrderDate'].dt.year
df['Month'] = df['OrderDate'].dt.month

df['TotalSales'] = df['Quantity'] * df['UnitPrice']
print(df['Year'].head(),df['Month'].head(),df['TotalSales'].head())


0    2023
1    2023
2    2022
3    2023
4    2023
Name: Year, dtype: int32 0     1
1    12
2     5
3     7
4     2
Name: Month, dtype: int32 0     319.77
1     251.37
2     105.09
3     167.90
4    1031.28
Name: TotalSales, dtype: float64


In [13]:
# Total sales by year
sales_by_year = df.groupby('Year')['TotalSales'].sum()
print(sales_by_year)

# Top 10 products by sales
top_products = df.groupby('ProductName')['TotalSales'].sum().sort_values(ascending=False).head(10)
print(top_products)


Year
2020    18355658.73
2021    18085812.35
2022    18197768.61
2023    18361035.17
2024    18020938.13
Name: TotalSales, dtype: float64
ProductName
Memory Card 128GB              1925774.79
LED Desk Lamp                  1909854.95
Mechanical Keyboard            1886484.85
Electric Kettle                1886239.46
Smartwatch                     1881961.28
Gaming Mouse                   1881177.65
Water Bottle                   1879086.56
Dress Shirt                    1872813.24
Noise Cancelling Headphones    1869793.13
Kids Toy Car                   1868347.34
Name: TotalSales, dtype: float64


In [14]:
# Check negative values
df[df['Quantity'] <= 0]

# Check duplicate orders
df[df.duplicated(subset='OrderID')]


,OrderID,OrderDate,CustomerID,CustomerName,ProductID,ProductName,Category,Brand,Quantity,UnitPrice,...,TotalAmount,PaymentMethod,OrderStatus,City,State,Country,SellerID,Year,Month,TotalSales


In [17]:
customer_summary = df.groupby('CustomerID').agg(
    total_orders=('OrderID', 'nunique'),
    total_sales=('TotalSales', 'sum'),
    avg_order_value=('TotalSales', 'mean')
).reset_index()

print(customer_summary.head())

   CustomerID  total_orders  total_sales  avg_order_value
0  CUST000001             1       453.36       453.360000
1  CUST000002             3      1394.75       464.916667
2  CUST000003             1      1206.85      1206.850000
3  CUST000004             3      2857.24       952.413333
4  CUST000005             1      1215.00      1215.000000


In [18]:
product_summary = df.groupby('ProductName').agg(
    total_quantity=('Quantity', 'sum'),
    total_sales=('TotalSales', 'sum')
).reset_index()
print(product_summary)

                    ProductName  total_quantity  total_sales
0                    4K Monitor            6111   1846679.66
1                 Action Camera            6104   1810180.17
2                     Air Fryer            5956   1778056.72
3                      Backpack            5848   1734316.40
4             Bluetooth Speaker            5782   1790041.97
5                    Board Game            6200   1848573.14
6                   Car Charger            5754   1760180.70
7               Children's Book            5619   1719460.52
8                  Cookware Set            6130   1860255.19
9                Desk Organizer            6079   1832361.24
10                   Desk Plant            5847   1751649.46
11                  Dress Shirt            5960   1872813.24
12                   Drone Mini            5975   1797089.37
13              Electric Kettle            6165   1886239.46
14             External HDD 2TB            5858   1777840.15
15                 Fitne

In [19]:
df.to_csv("amazon_orders_cleaned.csv", index=False)
customer_summary.to_csv("customer_summary.csv", index=False)
product_summary.to_csv("product_summary.csv", index=False)
